In [1]:
import sys
sys.path.append('/home/maximilian.hofmann/ff_utils/src')

from importlib import reload

In [2]:
from importlib import reload
from itertools import repeat
from date_utils import partition_date_range
import pandas as pd
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count
from datetime import datetime

In [3]:
import functions as fu
reload(fu)

/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_geopandas/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


<module 'functions' from '/home/maximilian.hofmann/tools/pricing-tool-v2/data/functions.py'>

In [4]:
from ds.utilities.io import ds_trino

In [5]:
sql = '''
SELECT DISTINCT 
    CAST(DATE_TRUNC('MONTH', last_seen) AS DATE) AS month,
    super_region,
    country_code,
    service_slug,
    COUNT(*)
FROM iceberg.marketintel.storefront_latest_view
GROUP BY 1,2,3,4
ORDER BY 2,1,3,4
'''

In [20]:
data = ds_trino.fetch_data(sql)

KeyboardInterrupt: 

In [9]:
data['country_code'].unique()

array(['AUS', 'TWN', None, 'JPN', 'MNG', 'NPL', 'NZL', 'PHL', 'IND',
       'CHN', 'GBR', 'CHE', 'DEU', 'ESP', 'FRA', 'ITA', 'BEL', 'LUX',
       'PRT', 'IRL', 'KGZ', 'NLD', 'RUS', 'TUR', 'CZE', 'KAZ', 'SWE',
       'MEX', 'ARG', 'CHL', 'DOM', 'USA', 'CAN', 'PRI', 'ATA'],
      dtype=object)

In [10]:
data[data['super_region']=='LATAM']

,month,super_region,country_code,service_slug,_col4
139,2022-06-01,LATAM,MEX,ubereats-marketintel,10
140,2022-07-01,LATAM,MEX,doordash-marketintel,1
141,2022-07-01,LATAM,MEX,grubhub-marketintel,8
142,2022-07-01,LATAM,MEX,ubereats-marketintel,504
143,2022-07-01,LATAM,None,ubereats-marketintel,4
144,2022-08-01,LATAM,MEX,doordash-marketintel,4
145,2022-08-01,LATAM,MEX,ubereats-marketintel,1636
146,2022-08-01,LATAM,None,ubereats-marketintel,13
147,2022-09-01,LATAM,MEX,doordash-marketintel,2
148,2022-09-01,LATAM,MEX,ubereats-marketintel,1719


In [6]:
begin_date = pd.to_datetime('2023-02-01')
end_date = datetime.now().strftime('%Y-%m-%d')
days_per_batch = 10

In [7]:
super_region = 'LATAM'
country_code = 'MEX'
service_slug = 'ubereats-marketintel'

In [14]:
with open('/home/maximilian.hofmann/tools/pricing-tool-v2/data/queries/sql_ofo_scrape_data_iceberg.sql', 'r') as f:
    sql_template = f.read()

In [9]:
project_name = 'css-operations'
stag_dataset_name = 'brand_science_stag'
stag_table_name = 'pricing_ofo_scrape_data_stag'
full_stag_table_name = '.'.join([project_name, stag_dataset_name, stag_table_name])

In [10]:
date_list = partition_date_range(pd.to_datetime(begin_date), pd.to_datetime(end_date), days_per_batch, week_trunc=False)
date_list

[['2023-02-01', '2023-02-10'],
 ['2023-02-11', '2023-02-20'],
 ['2023-02-21', '2023-02-27']]

In [11]:
clustering_fields=['country_code','month']

In [12]:
num_processes = min(cpu_count(), len(date_list))
num_processes

3

In [ ]:
res = []
with Pool(num_processes) as p:
    r = p.starmap(fu.process_ofo_data_date_range_iceberg, zip(repeat(sql_template), date_list, repeat(super_region), repeat(service_slug), repeat(full_stag_table_name), repeat(clustering_fields)))
    res.append(r)

Start data cleaning...
Start data cleaning...
Data cleaning finished!
Loading formatted geocoded file...
Job 0111f4cb-eca5-434d-bc24-e01f98a809a4 is currently in state RUNNING
Job 0111f4cb-eca5-434d-bc24-e01f98a809a4 is currently in state RUNNING
Job 0111f4cb-eca5-434d-bc24-e01f98a809a4 is currently in state RUNNING
Job 0111f4cb-eca5-434d-bc24-e01f98a809a4 is currently in state RUNNING
Job 0111f4cb-eca5-434d-bc24-e01f98a809a4 is currently in state RUNNING
Start data cleaning...


In [3]:
prod_dataset_name = 'brand_science'
prod_table_name = 'pricing_ofo_scrape_data'
full_prod_table_name = '.'.join([project_name, prod_dataset_name, prod_table_name])

In [ ]:
import time
time.sleep(60)

In [10]:
create_tbl = fu.create_tbl(full_stag_table_name, full_prod_table_name)

In [11]:
upsert_tbl = fu.upsert_tbl_ofo_scrape_data(full_stag_table_name, full_prod_table_name)